In [2]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mahotas

In [ ]:



# Load the image
image_path = "/home/duyle/Rice_photos/BC-15/positive/DSC6771_idx1.png"  # Update with your image path
image = cv2.imread(image_path)

# Convert to grayscale
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian Blur to reduce noise
blur = cv2.GaussianBlur(image, (3, 3), 0)
threshold, new_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contour = max(contours, key=cv2.contourArea)


    # Compute Hu Moments
moments = cv2.moments(contour)
hu_moments = cv2.HuMoments(moments)

# Apply log transformation with a small epsilon to prevent log(0) issues
hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)


print(hu_moments)

[ WARN:0@0.909] global loadsave.cpp:268 findDecoder imread_('/home/duyle/Rice_photos/BC-15/BC-15/DSC7436_idx47.png'): can't open/read file: check file path/integrity


error: OpenCV(4.11.0) /usr/src/debug/opencv/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
from scipy.ndimage import convolve
def extract_edge_histogram_features(image):
    """
    Compute a simplified edge histogram descriptor (EDH) from the image.
    The function applies five convolution kernels corresponding to different edge orientations.
    """
    # Convert to grayscale if necessary.
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    
    # Define convolution kernels for five edge types.
    kernels = {
        "vertical": np.array([[-1,  2, -1],
                              [-1,  2, -1],
                              [-1,  2, -1]], dtype=np.float32),
        "horizontal": np.array([[-1, -1, -1],
                                [ 2,  2,  2],
                                [-1, -1, -1]], dtype=np.float32),
        "diag_45": np.array([[-1, -1,  2],
                             [-1,  2, -1],
                             [ 2, -1, -1]], dtype=np.float32),
        "diag_135": np.array([[ 2, -1, -1],
                              [-1,  2, -1],
                              [-1, -1,  2]], dtype=np.float32),
        # A non-directional (Laplacian-like) kernel.
        "non_directional": np.array([[1,  1,  1],
                                     [1, -8,  1],
                                     [1,  1,  1]], dtype=np.float32)
    }
    
    features = {}
    total_energy = 0.0
    
    # Convolve the image with each kernel and compute the energy.
    for key, kernel in kernels.items():
        response = convolve(gray.astype(np.float32), kernel, mode="reflect")
        energy = np.sum(np.abs(response))
        features[f"edge_energy_{key}"] = energy
        total_energy += energy
    
    # Normalize to form a histogram (if total energy > 0).
    if total_energy > 0:
        for key in list(features.keys()):
            features[key] /= total_energy

    return features

In [ ]:
extract_edge_histogram_features(cv2.imread('/home/duyle/Rice_photos/BC-15/BC-15/DSC7436_idx70.png'))

{'edge_energy_vertical': np.float32(0.11537549),
 'edge_energy_horizontal': np.float32(0.246982),
 'edge_energy_diag_45': np.float32(0.16055416),
 'edge_energy_diag_135': np.float32(0.15892461),
 'edge_energy_non_directional': np.float32(0.31816378)}

In [ ]:
def color_structure_descriptor(image, grid_size=(8, 8), bins=32):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    
    height, width = hsv_image.shape[:2]
    cell_h, cell_w = height // grid_size[0], width // grid_size[1]

    
    hist_bins = [bins] 

    
    csd = []

    for row in range(grid_size[0]):
        for col in range(grid_size[1]):
            x_start, y_start = col * cell_w, row * cell_h
            x_end, y_end = x_start + cell_w, y_start + cell_h
            cell = hsv_image[y_start:y_end, x_start:x_end]

            hist = cv2.calcHist([cell], [0], None, hist_bins, [0, 180])
            hist = hist.flatten() 
            hist = hist / hist.sum() if hist.sum() != 0 else hist

            csd.extend(hist)

    return {f"csd_{i}":csd[i] for i in range(len(csd))}


In [ ]:
image_path = "/home/duyle/Rice_photos/BC-15/BC-15/DSC7436_idx47.png"  # Update with your image path
image = cv2.imread(image_path)
dic=  color_structure_descriptor(image)


In [4]:
df = pd.read_csv('/home/duyle/Rice_photos/features_extracted/moments/new_data_test_nep87.csv')

In [5]:
df

,Model,Dataset,Feature Combination,Accuracy,Precision,Recall,F1 Score,CV_scores(5fold)
0,K-Nearest Neighbors,Nep-87,Basic+Enhanced Color+Skewness and Kurtosis,0.9419,0.9432,0.9419,0.9419,0.9458
1,Support Vector Machine,Nep-87,Basic+Enhanced Color+Skewness and Kurtosis,0.9852,0.9853,0.9852,0.9852,0.9818
2,Random Forest,Nep-87,Basic+Enhanced Color+Skewness and Kurtosis,0.9736,0.9736,0.9736,0.9736,0.9729
3,K-Nearest Neighbors,Nep-87,Basic+Enhanced Color+Zernike moments,0.9609,0.9614,0.9609,0.9609,0.9740
4,Support Vector Machine,Nep-87,Basic+Enhanced Color+Zernike moments,0.9820,0.9821,0.9820,0.9820,0.9865
...,...,...,...,...,...,...,...,...
652,Support Vector Machine,Nep-87,Enhanced Color+Skewness and Kurtosis+Zernike m...,0.9863,0.9863,0.9863,0.9863,0.9891
653,Random Forest,Nep-87,Enhanced Color+Skewness and Kurtosis+Zernike m...,0.9641,0.9642,0.9641,0.9641,0.9682
654,K-Nearest Neighbors,Nep-87,Basic+Enhanced Color+Skewness and Kurtosis+Zer...,0.9556,0.9567,0.9556,0.9557,0.9734
655,Support Vector Machine,Nep-87,Basic+Enhanced Color+Skewness and Kurtosis+Zer...,0.9852,0.9852,0.9852,0.9852,0.9891
